# LLMSの応用(1)：Langchainの基礎

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/lvzeyu/css_nlp/blob/master/notebook/langchain_basic.ipynb)

[LangChain](https://www.langchain.com/)は大規模言語モデルを活用したアプリケーション開発を支えるフレームワークです。具体的にはプロンプト設計、RAGインデックス構築、会話メモリ、外部ツール連携、自律エージェントなどLLMアプリに必要な一連の機能を統合的に提供します。これにより、実験段階のプロトタイプから実運用を想定したシステムまで、開発を効率化しやすくなります。

- ```llms```: 言語モデルを呼び出すためのラッパーを提供します
- ```prompts```: プロンプトのテンプレートを作成する機能を提供します
- ```chains```: ひとつのワークフロー内で LLM やプロンプトテンプレートを組み合わせて使用するための機能を提供します
- ```agents```: エージェントを使用することで、課題の解決順序をも LLM を用いて決定し、実行させることができます
- ```memory```: チェーンとエージェントに状態を持たせるための機能を提供します

In [14]:
!pip install langchain==0.1.12
!pip install langchain-community
!pip install langchain-core
!pip install langsmith
!pip install openai
!pip install python-dotenv
!pip install langchain-openai
!pip install -U --quiet langchain-google-genai pillow
!pip install google-generativeai
!pip install google-search-results numexpr wikipedia langchain-experimental
!pip install duckduckgo-search

  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.4
    Uninstalling langsmith-0.6.4:
      Successfully uninstalled langsmith-0.6.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-split

## LangChainでLLMを使う


LLMは、ほとんどの生成AIアプリケーションを駆動する原動力である。LangChainは、あらゆるLLM APIプロバイダーとやり取りするためのシンプルなインターフェースを2種類提供している。

- チャットモデルインターフェース: チャットモデルインターフェースを使用すると、ユーザーとモデルの間で双方向の対話を行える
- LLMインターフェース: LLMインターフェースは、文字列プロンプトを入力として受け取り、それをモデルプロバイダーへ送信し、予測結果を出力として返すだけである。


[OpenAI](https://openai.com/blog/openai-api)と[Google(Gemini)](https://deepmind.google/technologies/gemini/#introduction)が提供するAPIを通じて、多岐にわたるAIモデルへのアクセスを可能になります。

APIを使うためには、まず「自分がサービスを利用できる証」となる「APIキー」を発行する必要があります。そして、OpenAIのAPIは、このAPIキーによって利用した使用量に応じて、課金される仕組みです。

そのため、APIキーが外部に漏れると、他者によって不正に使用されて料金が発生してしまうため、他の人へ共有しないように注意しましょう。

LangChainは、さまざまな LLM に汎用インターフェースを提供し、ユーザーがAPIを介してさまざまなモデルを操作できるようにします。

### OpenAI API

`OpenAI`クラスのインスタンスでGPT-3モデルを使用するための設定を行っています。

- `model_name`という引数で、使用する[モデルの名前](https://platform.openai.com/docs/models)を指定します。


- `temperature`という引数は、生成されるテキストのランダム性を制御します。`temperature`が高いほど（1に近いほど）、出力はよりランダムになります。逆に、`temperature`が低いほど（0に近いほど）、モデルの出力はより一貫性があり、予測可能になります。

- `max_tokens`という引数は、生成するテキストの最大トークン数を指定します。この場合、生成されるテキストは最大で256トークンになります。トークンとは、テキストを分割した単位のことで、一般的には単語や句読点などが1トークンとなります。


In [ ]:
from dotenv import load_dotenv
import os
import getpass
from langchain_openai import OpenAI, ChatOpenAI     

load_dotenv(dotenv_path='../.env')
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [16]:
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.9,
    max_tokens=512
)

In [17]:
llm.invoke("東北大学を紹介してください：")

'\n\n東北大学は日本の東北地方に位置する国立大学で、高い教育水準と優れた研究成果で知られています。1886年に創立され、現在は仙台市を中心に6つのキャンパスで約16,000人の学生が学ぶ総合大学です。\n\n東北大学には11学部・16研究科があり、理学、工学、経済学、法学、医学、歯学、薬学、農学、教育学、生命科学、国際文化研究を学ぶことができます。また、東北地方で唯一の獣医学部を有しており、幅広い分野での専門教育を提供しています。\n\n東北大学はグローバルな教育を重視し、多様な国・地域からの留学生や外国人教員を受け入れています。学生は国際交流プログラムや海外研修プログラムに参加し、世界各国での学びや経験を積むことができます。\n\nまた、東北大学は幅広い研究分野で国内外から高い評価を受けており、特に科学技術分野での研究成果は世界的に高い水準を誇っています。さまざまな国内外の企業や研究機関との共同研究も積極的に行われており、卒業生の就職率も非常に高いです。\n\n東北大学は東日本大震災の被災地に位置し'

チャットモデルインターフェースを使用すると、ユーザーとモデルの間で双方向の対話を行える。これが別のインターフェースとして用意されている理由は、OpenAIのような主要LLMプロバイダーが、送受信されるメッセージを「ユーザー」「アシスタント」「システム」というロールに分けて扱っているためである.

ここでのロールとは、メッセージに含まれるコンテンツの種類を示す）。

- システム（system）」ロール: モデルに与える指示を記述する。
- ユーザー（user）ロール: ユーザーのクエリおよびユーザーが生成したコンテンツを表す。
- アシスタント（assistant）ロール: モデルが生成したコンテンツを表す。

In [44]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI()
prompt = [HumanMessage("東北大学を紹介してください")]

model.invoke(prompt)


AIMessage(content='東北大学は日本の宮城県仙台市に位置する国立大学であり、東北地方における最高峰の総合大学です。1907年に創立され、現在は学部・大学院を含む12学部、13大学院があります。\n\n東北大学は、幅広い学問分野をカバーする「研究型大学」として知られており、工学、理学、医学、人文社会科学など多岐にわたる分野で高い研究力を持っています。特に理工学部や医学部は国内外から高い評価を受けており、多くの優秀な研究者を輩出しています。\n\nまた、東北大学は学生支援体制も充実しており、学生寮や奨学金制度、留学プログラムなど、学生が安心して学び、研究に取り組む環境が整っています。\n\n東北大学は、地域社会や産業界との連携を重視し、地域に貢献する研究活動やイノベーション創出にも積極的に取り組んでいます。その他にも、学内外の様々な国際交流プログラムやイベントが充実しており、多様な価値観や文化と触れ合う機会も提供されています。\n\n総合的に見て、東北大学は、高い学術水準と研究力、豊富な学生支援体制、そして社会貢献活動において、日本を代表する大学の一つとして、国内外で高い評価を受けています。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 531, 'prompt_tokens': 18, 'total_tokens': 549, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D0HhMdLc0IszUBjoF02omJBbL8Gu7', 's

In [45]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()         
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three exclamation marks.'''
)# あなたは優秀なアシスタントです。質問に対して3つの感嘆符を付けて回答してください。

human_msg = HumanMessage("東北大学を紹介してください")
model.invoke([system_msg, human_msg])


AIMessage(content='東北大学は日本の宮城県にある有名な大学です!!!東北地方で最も歴史と伝統のある大学の1つです!!!研究分野も豊富で、学生のレベルも高いです!!!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 37, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D0HluCvkJOuwacPEF7z9jj47ebzfZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bde45-f417-7481-b5b5-03794e351c03-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 77, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Gemini API

#### Gemini APIの設定


In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GOOGLE API key: ")

In [19]:
import google.generativeai as genai
models = [m for m in genai.list_models()]
models

/home/lvzeyu/anaconda3/envs/jupyterbook/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/tmp/ipykernel_152305/3039597036.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


[Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtain a distributed representation of a text.',
       input_token_limit=1024,
       output_token_limit=1,
       supported_generation_methods=['embedText', 'countTextTokens'],
       temperature=None,
       max_temperature=None,
       top_p=None,
       top_k=None),
 Model(name='models/gemini-2.5-flash',
       base_model_id='',
       version='001',
       display_name='Gemini 2.5 Flash',
       description=('Stable version of Gemini 2.5 Flash, our mid-size multimodal model that '
                    'supports up to 1 million tokens, released in June of 2025.'),
       input_token_limit=1048576,
       output_token_limit=65536,
       supported_generation_methods=['generateContent',
                                     'countTokens',
                                     'createCachedContent',
                                     'batc

In [20]:
import google.generativeai as genai

llm = genai.GenerativeModel('models/gemini-2.5-flash')

In [21]:
response = llm.generate_content("東北大学を紹介してください：")

In [22]:
from IPython.display import Markdown
Markdown(response.text)

東北大学は、日本を代表する国立大学の一つであり、特に旧帝国大学の一角として、国内外から非常に高い評価を受けています。宮城県仙台市に本部を置き、その歴史、理念、研究力、そして地域との結びつきにおいて、多くの魅力を持っています。

以下に東北大学の主な特徴を紹介します。

---

### 東北大学の主な特徴

1.  **歴史と伝統：旧帝国大学の一角**
    *   1907年に東北帝国大学として創立され、東京、京都に次ぐ3番目の帝国大学として設立されました。日本の高等教育、特に科学技術分野の発展に大きく貢献してきました。
    *   「旧帝大」としての揺るぎない地位は、その教育・研究水準の高さを示す証でもあります。

2.  **三つの基本理念：門戸開放、実学尊重、研究第一主義**
    *   **門戸開放 (Open-Door Policy)：** 創立当初から、出身地や家柄、性別にとらわれず、意欲と能力のある学生に広く門戸を開くという革新的な理念を掲げました。実際、日本で初めて女子学生と外国人留学生を受け入れた大学の一つとしても知られています。
    *   **実学尊重 (Emphasis on Practical Learning)：** 理論だけでなく、それが社会や産業にどう貢献できるかという「実学」を重んじています。基礎研究から応用研究まで、幅広く実践的な学問を追求しています。
    *   **研究第一主義 (Research First Principle)：** 教育とともに研究を大学の重要な使命と位置づけ、世界をリードする独創的な研究を推進しています。この理念のもと、多くの世界的な発見や革新的な技術が生まれています。

3.  **学術分野の多様性と研究力の高さ**
    *   理学、工学、医学、薬学、農学、文学、法学、経済学、教育学など、幅広い分野にわたる学部・研究科を有し、総合大学としての強みを持っています。
    *   特に、**材料科学、物理学、半導体関連技術、ロボット工学、医学・生命科学、災害科学**などの分野では世界的に高い研究水準を誇り、多くの研究機関（金属材料研究所、電気通信研究所、多元物質科学研究所など）が設置されています。
    *   スパコン「青葉」など、最先端の研究設備も充実しています。

4.  **国際性とグローバルな展開**
    *   「門戸開放」の理念は、国際性にもつながっています。世界各国の大学や研究機関との連携を強化し、多くの外国人留学生を受け入れています。
    *   国際共同研究や国際会議の開催にも積極的で、グローバルリーダーの育成にも力を入れています。

5.  **「学都仙台」の中心的存在**
    *   仙台市は「学都」として知られており、その中心に東北大学があります。地域社会との連携も深く、産学連携プロジェクト、市民向けの公開講座、地域貢献活動などを通じて、地域の活性化に貢献しています。
    *   豊かな自然と都市機能が調和した仙台の環境は、学生にとって恵まれた学習・生活環境を提供しています。

6.  **美しいキャンパスと充実した学生生活**
    *   広大なキャンパスには、歴史的建造物と最新の研究施設が共存しています。特に青葉山キャンパスは自然豊かで、学びに集中できる環境です。
    *   学園祭「東北大学祭」をはじめとする学生イベント、多様なサークル活動が盛んで、充実した学生生活を送ることができます。

---

東北大学は、その伝統に裏打ちされた高い研究力と教育力、そして常に未来を見据える革新的な精神で、国内外から優秀な人材が集まる魅力的な学府です。研究者を目指す方、社会をリードする人材になりたい方にとって、非常に魅力的な選択肢となるでしょう。

## LLMプロンプトを再利用する: Prompt Templates

プロンプトがモデルの出力に大きく影響することを示した。プロンプトはモデルに文脈を理解させ、クエリに対する適切な回答を生成させる助けとなります。

プロンプトは一見単なる文字列であるが、どのような内容を含めるべきか、またユーザー入力に応じてどのように変化させるべきかを設計するのは容易ではない。

プロンプトテンプレートは、プロンプトを作成する再現可能な方法を指します。これには、エンドユーザーから一連のパラメーターを受け取り、プロンプトを生成するテキスト文字列(テンプレート)が含まれます。

テンプレートは動的パラメータを挿入する位置の定義することで、静的かつ具体的なプロンプトを生成するレシピとして利用できます。

In [23]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["language","text"],
    template="次の日本語のテキストを{language}に翻訳してください：{text}",
)


In [24]:
print(prompt.format(language="英語", text="東北大学は日本の東北地方にある大学です。"))

次の日本語のテキストを英語に翻訳してください：東北大学は日本の東北地方にある大学です。


## チェーン

### チェーンの基本

LLM は単独でも十分に強力に機能します。 しかし、 LLM 同士を組合わせたり、ある機能に特化した他のモジュールとともに利用することで、より複雑なアプリケーションを構築することができます。 LangChain では、このような他の機能と連結するための汎用的なインターフェースとして、チェーンを提供しています。 チェーンを用いることで、LLM の利用を含む "一連の処理" を一つのまとまりとして扱うことができます。

つまり、平易な言葉でいえば、チェーンは「複数の処理の連なり」です。 この処理の連鎖の部品となるチェーンの構成要素のことを リンク と呼びます。 リンクの一例は、LLM の呼び出しなどの基本的な処理です。さらには、その他のチェーン全体をリンクとして含むチェーンも作成できます。

チェーンの代表例は、LLM とプロンプトテンプレートを組合わせて使用するための```LLMChain```です。 このチェーンを用いると、

- ユーザーの入力を受け取り
- それをPromptTemplateでフォーマットし
- フォーマットされたレスポンスを LLM に渡す

という一連の操作を一つのまとまりとして実行できます。 

基本的な使い方としては、LLM や プロンプトテンプレートなどの基本要素を組み合わせて使用することが考えられます。

In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [26]:
chain = prompt | llm
chain.invoke(
    {"language":"英語", "text":"東北大学は日本の東北地方にある大学です。"}
)

AIMessage(content='Tohoku University is a university in the Tohoku region of Japan.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde36-c7e5-7563-9c0b-43972be9772f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 21, 'output_tokens': 41, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 27}})

In [27]:
chain.invoke({"language":"中国語", "text":"東北大学は日本の東北地方にある大学です。"})

AIMessage(content='東北大学是位于日本东北地方的一所大学。', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde36-d7b9-7cb3-a333-3702e5edb5c2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 22, 'output_tokens': 593, 'total_tokens': 615, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 582}})

### 実装例:Few Shot Learning

In [28]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate


In [29]:
examples = [
    {"word": "楽しい", "antonym": "悲しい"},
    {"word": "高い", "antonym": "低い"},
]

In [30]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)


- `examples`: モデルに示す例を指定します。
- `example_prompt`: 例をどのように提示するかを指定します。
- `prefix`: プロンプトの前置詞を指定します。一般的には、モデルにタスクを説明するためのものです。
- `suffix`: プロンプトの後置詞を指定します。一般的には、モデルに入力と出力の形式を示すためのものです。
- `input_variables`: 入力の変数名を指定します。
- `example_separator`: 例を区切るための文字列を指定します。

In [31]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

In [32]:
print(few_shot_prompt.format(input="大きい"))

Give the antonym of every input

Word: 楽しい
Antonym: 悲しい



Word: 高い
Antonym: 低い


Word: 大きい
Antonym:


In [33]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [34]:
from langchain_core.output_parsers import StrOutputParser

chain = few_shot_prompt | llm | StrOutputParser()

var = few_shot_prompt.input_variables[0] 
print(chain.invoke({var: "大きい"}))

小さい


#### 課題

Few Shot Learningでセンチメント分析を実装しなさい。

- データフレームから一部のテキストとラベル($n=5$)を抽出し、`examples`を作成します
- Few Shot Learningためのpromptを作成します
- chainを作成し、テストデータから任意のテキストに対するセンチメント予測結果を確認します

In [35]:
#!pip install datasets
from datasets import load_dataset
dataset = load_dataset("imdb")

In [36]:
df_train_sample = dataset["train"].to_pandas().sample(5, random_state = 123)
df_test_sample = dataset["test"].to_pandas().sample(5, random_state = 123)

In [37]:
df_train_sample

,text,label
20000,After reading some quite negative views for th...,1
5515,Really no reason to examine this much further ...,0
966,"""Happy Go Lovely"" has only two things going fo...",0
22726,This movie is the first of Miikes triad societ...,1
2690,"Normally I would never rent a movie like this,...",0
